In [2]:
from collections import deque


DIRECT = True
REVERSE = False
# print(full_msg)

def label_content_bound(label: str, content: str, margin = 2):
    return max(len(label), len(content)) + margin


def label_and_content(label: str, content, filling: str = ' ') -> tuple[str, str]:
    printed_content = filling.join(str(bit) for bit in content)
    bound = label_content_bound(label, printed_content)
    label = label.center(bound)
    printed_content = printed_content.center(bound)
    return label, printed_content


def print_labels_and_content(full_msg: deque, state: deque, out_res: list) -> None:
    signal_label, printed_signal = label_and_content('Сигнал:', reversed(full_msg))
    state_label, printed_state = label_and_content('Регистры:', state, '|-|')
    out_label, printed_out = label_and_content('Out:', out_res[::-1])

    print(signal_label + state_label + out_label)
    print(printed_signal + printed_state + printed_out)
    print('')


def register_multiplication(n: int, q: int, begin_state: list, out_poly: list, msg: list):
    state = deque(begin_state)
    full_msg = deque(msg + [0] * max(n, len(begin_state), len(out_poly)))
    full_len = len(full_msg)
    out_res = []
    leftovers = []
    
    print('Начальное состояние:')
    print_labels_and_content(full_msg, state, out_res)
    
    for i in range(1, full_len):
        new_signal = full_msg[0]
        full_msg.popleft()
        state.appendleft(new_signal)
        leftovers.append(state.pop())
        to_out = sum([look * pos for look, pos in zip(out_poly, state)]) % q
        out_res.append(to_out)
        print(f'Шаг {i}:')
        print_labels_and_content(full_msg, state, out_res)

In [3]:
def check_length(left, right, label: str):
    if len(left) != len(right):
        raise ValueError(f'Args of different length in {label}: {left} and {right}')


def GF_mul(left, right, q: int):
    check_length(left, right, 'GF_mul')
    return sum([l * r for l, r in zip(left, right)]) % q


def GF_sum(left: int, right: int, q: int):
    return (left + right) % q


def register_division(n: int, q: int, begin_state: list[int], out_poly: list[int], den: list[int], msg: list[int], max_len: int = 10, leftover_out = False):
    len_den = len(den)
    len_begin = len(begin_state)
    
    if len_den - 1 > len_begin:
        state = deque(begin_state + [0] * (len(den) - len(begin_state) - 1))
    elif len_begin + 1 > len_den:
        den = den.copy()
        den = den + [0] * (len_begin - len_den + 1)
        state = deque(begin_state)
    else:
        state = deque(begin_state)
    full_msg = deque(msg + [0] * max_len)
    len_state = len(state)
    len_out_poly = len(out_poly)
    if len_state > len_out_poly:
        out_poly = out_poly.copy()
        out_poly = out_poly + [0] * (len_state - len_out_poly)
    elif len_state < len_out_poly:
        state = state + [0] * (len_out_poly - len_state)
    full_len = len(full_msg)
    out_res = []
    leftovers = []
    target = [i for i, x in enumerate(den) if x][0]
    defer = [i for i, x in enumerate(out_poly) if x][0]
    den = den.copy()
    den[target] = 0
    print('Начальное состояние:')
    print_labels_and_content(full_msg, state, out_res)
    # print(defer)
    for i in range(0, full_len):
        new_signal = full_msg[0]
        full_msg.popleft()
        state.appendleft(new_signal)
        state[target] = GF_sum(state[target], GF_mul(den, state, q), q)
        leftover = state.pop()
        leftovers.append(leftover)
        if i >= defer:
            to_out = GF_mul(out_poly, state, q)
            out_res.append(to_out)
        print(f'Шаг {i + 1}:')
        if leftover_out:
            print_labels_and_content(full_msg, state, leftovers)
        else:
            print_labels_and_content(full_msg, state, out_res)

## 9 ДЗ

### Первое задание

In [4]:
def GF_vec_sum(l, r, q: int) -> list[int]:
    return [(l_ + r_) % q for l_, r_ in zip(l, r)]


def GF_scalar_vec_mul(num: int, v, q: int) -> list[int]:
    return [num * v_ % q for v_ in v]


def state_to_int(state: deque[int], q: int) -> int:
    return sum(s * (q ** i) for i, s in enumerate(state))


def states_to_ints(states: list[deque[int]], q: int) -> list[int]:
    return [state_to_int(s, q) for s in states]


def register_self_feed(q: int, begin_state: list[int], den: list[int], elem_count: int = 10):
    state = deque(begin_state)
    full_msg = deque([0] * elem_count)
    full_len = len(full_msg)
    out_res = []
    leftovers = []
    target = [i for i, x in enumerate(den) if x][-1]
    den = den.copy()
    den[target] = 0
    print('Начальное состояние:')
    print_labels_and_content(full_msg, state, leftovers)
    states = []
    states.append(state.copy())
    # print(f'Appended {state_to_int(state, q)}')
    # print(defer)
    for i in range(0, full_len):
        new_signal = full_msg[0]
        full_msg.popleft()
        state.appendleft(new_signal)
        leftover = state[-1]
        state = deque(GF_vec_sum(state, GF_scalar_vec_mul(leftover, den, q), q))
        # print(f'Redequed state: {state}')
        state.pop()
        # print(f'Poppped state: {state}')
        states.append(state.copy())
        # print(f'Appended {state_to_int(state, q)}')
        # state[target] = GF_sum(state[target], GF_mul(den, state, q), q)
        leftovers.append(leftover)
        # to_out = GF_mul(out_poly, state, q)
        # out_res.append(to_out)
        print(f'Шаг {i + 1}:')
        print_labels_and_content(full_msg, state, leftovers)
    return states


def check_matrix(p: list[int], width: int) -> list[list[int]]:
    rev = p[::-1]
    len_p = len(p)
    return [
        [0] * i + rev + [0] * (width - len_p - i)
        for i in range(width - len_p + 1)
    ]

In [5]:
from pprint import pprint


n = 4
q = 2
begin_state = [0, 0, 1, 1]
numerator = [1]
denominator = [1, 1, 0, 0, 1]
msg = [0]

states = register_self_feed(q, begin_state, denominator, 14)
# print('States: ', states)
print(f'Achieved states: {sorted(states_to_ints(states, q))}')
H = check_matrix(denominator, q ** n - 1)
print('Check matrix:')
pprint(H)
# register_division(n, q, begin_state, numerator, denominator, msg, 14)

Начальное состояние:
           Сигнал:              Регистры:    Out: 
 0 0 0 0 0 0 0 0 0 0 0 0 0 0  0|-|0|-|1|-|1       

Шаг 1:
          Сигнал:             Регистры:    Out: 
 0 0 0 0 0 0 0 0 0 0 0 0 0  1|-|1|-|0|-|1   1   

Шаг 2:
         Сигнал:            Регистры:    Out: 
 0 0 0 0 0 0 0 0 0 0 0 0  1|-|0|-|1|-|0  1 1  

Шаг 3:
        Сигнал:           Регистры:     Out: 
 0 0 0 0 0 0 0 0 0 0 0  0|-|1|-|0|-|1  0 1 1 

Шаг 4:
       Сигнал:          Регистры:      Out:  
 0 0 0 0 0 0 0 0 0 0  1|-|1|-|1|-|0  1 0 1 1 

Шаг 5:
      Сигнал:         Регистры:       Out:   
 0 0 0 0 0 0 0 0 0  0|-|1|-|1|-|1  0 1 0 1 1 

Шаг 6:
     Сигнал:        Регистры:        Out:    
 0 0 0 0 0 0 0 0  1|-|1|-|1|-|1  1 0 1 0 1 1 

Шаг 7:
    Сигнал:       Регистры:         Out:     
 0 0 0 0 0 0 0  1|-|0|-|1|-|1  1 1 0 1 0 1 1 

Шаг 8:
   Сигнал:      Регистры:          Out:      
 0 0 0 0 0 0  1|-|0|-|0|-|1  1 1 1 0 1 0 1 1 

Шаг 9:
  Сигнал:     Регистры:           Out:       
 0 0 0 0 0  1|-

### Второе задание

In [6]:
def alpha_pair(alphas: list[list[int]], i: int, mod: int) -> list[int]:
    # print(f'Pair index = {(i * 3) % mod}')
    return alphas[(i * 3) % mod]


def form_matrix(alphas: list[list[int]]) -> list[list[int]]:
    return [[alpha[::-1][i] for alpha in alphas] for i in range(len(alphas[0]))]


def H_paired(alphas: list[list[int]], pairs: list[list[int]]) -> list[list[int]]:
    return [*form_matrix(alphas), *form_matrix(pairs)]

In [7]:
n = 4
q = 2
mod = q ** n - 1
begin_state = [1, 0, 0, 0]
denominator = [1, 1, 0, 0, 1]
msg = [0]

alphas = [list(state) for state in register_self_feed(q, begin_state, denominator, 14)]
# print(len(alphas))
# print(alphas)
alpha_pairs = [alpha_pair(alphas, i, mod) for i in range(len(alphas))]
H_double = H_paired(alphas, alpha_pairs)
print('Double H:')
pprint(H_double)

Начальное состояние:
           Сигнал:              Регистры:    Out: 
 0 0 0 0 0 0 0 0 0 0 0 0 0 0  1|-|0|-|0|-|0       

Шаг 1:
          Сигнал:             Регистры:    Out: 
 0 0 0 0 0 0 0 0 0 0 0 0 0  0|-|1|-|0|-|0   0   

Шаг 2:
         Сигнал:            Регистры:    Out: 
 0 0 0 0 0 0 0 0 0 0 0 0  0|-|0|-|1|-|0  0 0  

Шаг 3:
        Сигнал:           Регистры:     Out: 
 0 0 0 0 0 0 0 0 0 0 0  0|-|0|-|0|-|1  0 0 0 

Шаг 4:
       Сигнал:          Регистры:      Out:  
 0 0 0 0 0 0 0 0 0 0  1|-|1|-|0|-|0  1 0 0 0 

Шаг 5:
      Сигнал:         Регистры:       Out:   
 0 0 0 0 0 0 0 0 0  0|-|1|-|1|-|0  0 1 0 0 0 

Шаг 6:
     Сигнал:        Регистры:        Out:    
 0 0 0 0 0 0 0 0  0|-|0|-|1|-|1  0 0 1 0 0 0 

Шаг 7:
    Сигнал:       Регистры:         Out:     
 0 0 0 0 0 0 0  1|-|1|-|0|-|1  1 0 0 1 0 0 0 

Шаг 8:
   Сигнал:      Регистры:          Out:      
 0 0 0 0 0 0  1|-|0|-|1|-|0  1 1 0 0 1 0 0 0 

Шаг 9:
  Сигнал:     Регистры:           Out:       
 0 0 0 0 0  0|-

In [8]:
import numpy as np


c = [int(s) for s in list('111001100011011')]
q = 2
n = 4
i, j = 2, 9
e = c.copy()
e[i] = (e[i] + 1) % q
e[j] = (e[j] + 1) % q
# print(e)
e = np.array(e)
print(f'{e = }')
H_double = np.array(H_double)
print(f'{H_double = }')
syndrome = np.dot(e, H_double.T) % q
print(syndrome)

e = array([1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1])
H_double = array([[0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1]])
[1 1 1 0 0 0 1 1]


In [9]:
def calculate_sigma2(s1: np.ndarray, s2: np.ndarray, q: int) -> np.ndarray:
    # res = 
    sigma2 = ((np.polydiv(s2, s1)[0] % q) - (np.polymul(s1, s1) % q)) % q
    print(f'{sigma2}')
    return sigma2

In [10]:
s1, s2 = syndrome[:n], syndrome[n:]
s1_i = [i for i, alpha in enumerate(alphas) if alpha == list(s1)[::-1]][0]
s2_i = [i for i, alpha in enumerate(alphas) if alpha == list(s2)[::-1]][0]
# print(s1, s2)
# print(s1_i, s2_i)
alphas_ = np.array([alpha[::-1] for alpha in alphas])
# print(alphas_[(mod + s2_i - s1_i) % mod], alphas_[s1_i * 2 % mod])
sigma1 = s1
sigma2 = (alphas_[(mod + s2_i - s1_i) % mod] + alphas_[s1_i * 2 % mod]) % q
# print(f'{sigma1 = }, {sigma2 = }')
solutions = []
for i, alpha_ in enumerate(alphas_):
    res = np.polyadd(
        np.polyadd(
            np.polymul(alpha_, alpha_),
            np.polymul(sigma1, alpha_)
        ),
        sigma2
    )
    res = np.polydiv(res, denominator[::-1])[1] % q
    res = np.pad(res, (n - len(res), 0), 'constant', constant_values=(0, 0))
    print(f'{alpha_ = }, {res = }')
    if (res == 0).all():
        solutions.append(i)
print(f'{solutions = }')

alpha_ = array([0, 0, 0, 1]), res = array([0., 0., 0., 1.])
alpha_ = array([0, 0, 1, 0]), res = array([0., 1., 0., 1.])
alpha_ = array([0, 1, 0, 0]), res = array([0., 0., 0., 0.])
alpha_ = array([1, 0, 0, 0]), res = array([1., 0., 1., 1.])
alpha_ = array([0, 0, 1, 1]), res = array([1., 0., 1., 0.])
alpha_ = array([0, 1, 1, 0]), res = array([1., 0., 1., 1.])
alpha_ = array([1, 1, 0, 0]), res = array([0., 1., 0., 1.])
alpha_ = array([1, 0, 1, 1]), res = array([1., 1., 1., 1.])
alpha_ = array([0, 1, 0, 1]), res = array([1., 1., 1., 1.])
alpha_ = array([1, 0, 1, 0]), res = array([0., 0., 0., 0.])
alpha_ = array([0, 1, 1, 1]), res = array([0., 1., 0., 0.])
alpha_ = array([1, 1, 1, 0]), res = array([1., 1., 1., 0.])
alpha_ = array([1, 1, 1, 1]), res = array([0., 0., 0., 1.])
alpha_ = array([1, 1, 0, 1]), res = array([1., 0., 1., 0.])
alpha_ = array([1, 0, 0, 1]), res = array([0., 1., 0., 0.])
solutions = [2, 9]


### ДЗ 10

#### Проверка матриц

In [11]:
n = 4
q = 2
begin_state = [0, 0, 0, 0, 0]
out_poly = [1, 1, 0, 0, 1]
msg = [1, 1, 1, 1, 1]
register_multiplication(n, q, begin_state, out_poly, msg)

Начальное состояние:
       Сигнал:            Регистры:      Out: 
 0 0 0 0 0 1 1 1 1 1  0|-|0|-|0|-|0|-|0       

Шаг 1:
      Сигнал:           Регистры:      Out: 
 0 0 0 0 0 1 1 1 1  1|-|0|-|0|-|0|-|0   1   

Шаг 2:
     Сигнал:          Регистры:      Out: 
 0 0 0 0 0 1 1 1  1|-|1|-|0|-|0|-|0  0 1  

Шаг 3:
    Сигнал:         Регистры:       Out: 
 0 0 0 0 0 1 1  1|-|1|-|1|-|0|-|0  0 0 1 

Шаг 4:
   Сигнал:        Регистры:        Out:  
 0 0 0 0 0 1  1|-|1|-|1|-|1|-|0  0 0 0 1 

Шаг 5:
  Сигнал:       Регистры:         Out:   
 0 0 0 0 0  1|-|1|-|1|-|1|-|1  1 0 0 0 1 

Шаг 6:
 Сигнал:      Регистры:          Out:    
 0 0 0 0  0|-|1|-|1|-|1|-|1  0 1 0 0 0 1 

Шаг 7:
 Сигнал:      Регистры:           Out:     
  0 0 0   0|-|0|-|1|-|1|-|1  1 0 1 0 0 0 1 

Шаг 8:
 Сигнал:      Регистры:            Out:      
   0 0    0|-|0|-|0|-|1|-|1  1 1 0 1 0 0 0 1 

Шаг 9:
 Сигнал:      Регистры:             Out:       
    0     0|-|0|-|0|-|0|-|1  1 1 1 0 1 0 0 0 1 



In [12]:
def shift_matrix(p: list[int], width: int) -> list[list[int]]:
    cpy = p[:]
    len_p = len(p)
    return [
        [0] * i + cpy + [0] * (width - len_p - i)
        for i in range(width - len_p + 1)
    ]

In [13]:
g_x = [1, 0, 0, 0, 1, 0, 1, 1, 1]
# g_x = g_x[::-1]
G = np.array(shift_matrix(g_x, mod))
print('G:')
print(G)
print('G *H^T:')
print(G @ H_double.T % q)

G:
[[1 0 0 0 1 0 1 1 1 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 1 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 1 1 1 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 1 1 1 0 0]
 [0 0 0 0 0 1 0 0 0 1 0 1 1 1 0]
 [0 0 0 0 0 0 1 0 0 0 1 0 1 1 1]]
G *H^T:
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


#### Задание 2

In [16]:
def num_to_poly(num: int, base: int) -> np.ndarray:
    poly = []
    res = num
    while res > 0:
        poly.append(res % base)
        res //= base
    return np.array(poly)


def poly_str(poly: np.ndarray) -> str:
    return ' + '.join([str(poly[0])] + [f'{c}x^{i + 1}' for i, c in enumerate(poly[1:]) if c > 0])


def poly_to_num(poly: np.ndarray, base: int) -> int:
    return np.sum(np.dot(poly, [base ** p for p in range(len(poly))]))


def mul_poly_mod(l: np.ndarray, r: np.ndarray, mod_poly: np.ndarray, base: int):
    res = np.array(np.polydiv(np.polymul(l[::-1], r[::-1]) % base, mod_poly[::-1])[1][::-1] % base, dtype=int)
    # print(f'({l})*({r}) =(mod) {res}')
    return res


def get_base_poly_pow(poly: np.ndarray, mod_poly: np.ndarray, pow: int, base: int, start_p: int = 2):
    cur = poly
    print(f'Push x^{start_p} = {poly_str(poly)} to power {pow}')
    for i in range(1, pow - start_p + 1):
        prev = cur
        cur = np.polydiv(np.polymul(cur[::-1], [1, 0]), mod_poly[::-1])[1][::-1] % base
        print(f'x^{start_p + i} = {poly_str(cur)}')
    # res = np.polydiv(np.polymul(poly[::-1], ([0] * (pow - start_p) + [1])[::-1]), mod_poly[::-1])[1][::-1]
    # print(f'to pow = {res}')
    return cur


def get_orders(base: int, n: int, print_intermediate: bool = False):
    q = base ** n
    mod = q - 1
    divisors = [i for i in range(1, q) if mod % i == 0]
    buckets = {div: [] for div in divisors}

    for e in range(1, q):
        if print_intermediate:
            print(f'X^s = X^{e}')
        msg = ''
        for div in divisors:
            res = e * div % mod
            if res == 0:
                msg += f'X^({e}*{div}) = X^{res}'
                if print_intermediate:
                    print(msg)
                buckets[div].append(e % mod)
                break
            else:
                msg += f'X^({e}*{div}) = X^{res}; '
    return buckets

buckets = get_orders(2, 4)
print('Orders:')
print(buckets)

Orders:
{1: [0], 3: [5, 10], 5: [3, 6, 9, 12], 15: [1, 2, 4, 7, 8, 11, 13, 14]}
